In [1]:
# fix for memory issues
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from hw2.transforms import corrupt_mnist_copy

(x_train, _), (x_test, _) = mnist.load_data()

x_train_corrupted = corrupt_mnist_copy(x_train)
x_test_corrupted = corrupt_mnist_copy(x_test)

# Scale to [0, 1]
x_train = x_train.astype(K.floatx()) / 255.
x_train_corrupted = x_train_corrupted.astype(K.floatx()) / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_train_corrupted = np.reshape(x_train_corrupted, (len(x_train_corrupted), 28, 28, 1))

x_test = x_test.astype(K.floatx()) / 255.
x_test_corrupted = x_test_corrupted.astype(K.floatx()) / 255.
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
x_test_corrupted = np.reshape(x_test_corrupted, (len(x_test_corrupted), 28, 28, 1))

In [2]:
# Model a CNN autoencoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, SpatialDropout2D

input_img = Input(shape=(28,28,1))

x = Conv2D(32, [3,3], 
           strides=(1, 1), 
           padding='same',
           data_format='channels_last',
           use_bias=True,
           kernel_initializer='lecun_normal',
           bias_initializer='lecun_normal',
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           kernel_constraint=None,
           bias_constraint=None
          )(input_img)

x = Activation('relu')(x)

x = MaxPooling2D(pool_size=(2, 2), 
                 strides=None, 
                 padding='same', 
                 data_format=None)(x)

x = Conv2D(16, [3,3], 
           strides=(1, 1), 
           padding='same',
           data_format='channels_last',
           use_bias=True,
           kernel_initializer='lecun_normal',
           bias_initializer='lecun_normal',
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           kernel_constraint=None,
           bias_constraint=None
          )(x)

x = Activation('relu')(x)

x = MaxPooling2D(pool_size=(2, 2), 
                 strides=None, 
                 padding='same', 
                 data_format=None)(x)

encoded = Conv2D(16, [3,3], 
           strides=(1, 1), 
           padding='same',
           data_format='channels_last',
           use_bias=True,
           kernel_initializer='lecun_normal',
           bias_initializer='lecun_normal',
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           kernel_constraint=None,
           bias_constraint=None
          )(x)

x = Activation('relu')(encoded)

x = UpSampling2D(size=(2, 2), 
                 data_format='channels_last', 
                 interpolation='nearest'
                )(x)

x = Conv2D(32, [3,3], 
           strides=(1, 1), 
           padding='same',
           data_format='channels_last',
           use_bias=True,
           kernel_initializer='lecun_normal',
           bias_initializer='lecun_normal',
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           kernel_constraint=None,
           bias_constraint=None
          )(x)

x = Activation('relu')(x)

x = UpSampling2D(size=(2, 2), 
                 data_format='channels_last', 
                 interpolation='nearest'
                )(x)

decoder = Conv2D(1, [3,3], 
           strides=(1, 1), 
           padding='same',
           data_format='channels_last',
           activation='sigmoid',
           use_bias=True,
           kernel_initializer='lecun_normal',
           bias_initializer='lecun_normal',
           kernel_regularizer=None,
           bias_regularizer=None,
           activity_regularizer=None,
           kernel_constraint=None,
           bias_constraint=None
          )(x)

autoencoder = Model(input_img,decoder)

autoencoder.summary()

#adam = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        4624      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0     

In [3]:
%load_ext jupyternotify

from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler

tensor_board = TensorBoard('logs\\run4-32-16-16-32')

batch_size = 32
epochs = 20

def lr_scheduler(epoch, lr):
    if (epoch >= 0 and epoch < 10):
        return 0.001
#     elif (epoch >= 50 and epoch < 70):
#         return 0.0005
#     elif (epoch >= 70 and epoch < 80):
#         return 0.0001
#     elif (epoch >= 80 and epoch < 90):
#         return 0.00001
#     elif (epoch >= 90 and epoch < 100):
#         return 0.000001
    else:
        return 0.0001

callbacks = [tensor_board,
            LearningRateScheduler(lr_scheduler, verbose=1)
            ]

history = autoencoder.fit(x_train_corrupted, x_train, batch_size = batch_size, epochs=epochs, verbose=1,
         validation_data=(x_test_corrupted, x_test), callbacks=callbacks)

score = autoencoder.evaluate(x_train_corrupted, x_train,
                       batch_size=batch_size,
                       verbose=1, 
                       sample_weight=None, 
                       steps=None)

autoencoder.save('domingo_cnn_ae.h5') 

#print('Training took {} seconds'.format(time.time()-start_time))

print_score = f"{score[1]*100:.2f}%"
%notify -m "Training has finished. Validation accuracy is {print_score}."

<IPython.core.display.Javascript object>

Train on 60000 samples, validate on 10000 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/20
60000/60000 [==============================] - 10s 159us/sample - loss: 0.1350 - acc: 0.8070 - val_loss: 0.1161 - val_acc: 0.8081

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/20
60000/60000 [==============================] - 8s 138us/sample - loss: 0.1146 - acc: 0.8100 - val_loss: 0.1102 - val_acc: 0.8105

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Epoch 3/20
60000/60000 [==============================] - 8s 139us/sample - loss: 0.1100 - acc: 0.8106 - val_loss: 0.1066 - val_acc: 0.8092

Epoch 00004: LearningRateScheduler reducing learning rate to 0.001.
Epoch 4/20
60000/60000 [==============================] - 9s 144us/sample - loss: 0.1076 - acc: 0.8108 - val_loss: 0.1077 - val_acc: 0.8119

Epoch 00005: LearningRateScheduler reducing learning rate to 0.001.
Epoch 5/20
60000/60000 [====================

<IPython.core.display.Javascript object>

In [4]:
import matplotlib.pyplot as plt

output = autoencoder.predict(x_test_corrupted, verbose=1)

plt.imshow(np.squeeze(x_test_corrupted[50]),cmap='gray')
plt.show()
plt.imshow(np.squeeze(output[50]),cmap='gray')
plt.show()

10000/10000 [==============================] - 1s 57us/sample


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [6]:
# Benchmarking code

from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from hw2.benchmark_keras import test_model

(x_train2, y_train2), (x_test2, y_test2) = mnist.load_data()

# Scale to [0, 1]
x_test2 = x_test2.astype(K.floatx()) / 255.

# model is your Keras model
# DO NOT convert y_test, i.e. don't use keras.utils.to_categorical()
test_model(autoencoder, x_test2, y_test2, batch_size=100)

%notify -m "Training has finished."

Classifier score: 95.06
SSIM score: 97.17


<IPython.core.display.Javascript object>